In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

2024-10-27 20:34:04.078653: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-27 20:34:04.126687: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-27 20:34:05.012135: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
file_paths = [
    '/home/anudeep/devel/workspace/src/data/go1_eval/go1_trot_data_eval.csv'
]

dfs = [pd.read_csv(file_path) for file_path in file_paths]
df_combined = pd.concat(dfs, ignore_index=True)

X = df_combined.iloc[:, :-12]
Y = df_combined.iloc[:, -12:]

X = X.drop('time', axis=1)

print(X.shape)
print(Y.shape)

(15000, 44)
(15000, 12)


In [10]:
# Store the last 4 columns 
X_last_4 = X.iloc[:, -4:]

# scale the data to be between -1 and 1

X_min = X.iloc[:, :-4].min(axis= 0)
X_max = X.iloc[:, :-4].max(axis= 0)
Y_min = Y.min(axis= 0)
Y_max = Y.max(axis= 0)

X_norm = X.iloc[:, :-4]

X_norm = 2 * (X_norm - X_min) / (X_max - X_min) - 1
Y_norm = 2 * (Y - Y_min) / (Y_max - Y_min) - 1

X_input = pd.concat([X_norm, X_last_4], axis=1)





In [11]:
import time
import numpy as np
import torch
from END2ENDPredictor import NMPCPredictor 
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model
model_path = "/home/jovyan/userdata/ssajja2s/models/end2end_predictor.pth"
estimator = torch.load(model_path, map_location=device)  # Load model to the correct device

# Load the input data
# data_path = "/home/jovyan/userdata/ssajja2s/data_files/eval_data/go1_trot_data_eval.csv"
data_path = "/home/jovyan/userdata/ssajja2s/data_files/new_data/go1_trot_data_06.csv"
df = pd.read_csv(data_path)

# Prepare input and output data
X = df.iloc[:, :-12]
Y = df.iloc[:, -12:]

# Drop the first column 'time'
X = X.drop('time', axis=1)

# Store the last 4 columns 
X_last_4 = X.iloc[:, -4:]

# Scale the data to be between -1 and 1
X_min = X.iloc[:, :-4].min(axis=0)
X_max = X.iloc[:, :-4].max(axis=0)
Y_min = Y.min(axis=0)
Y_max = Y.max(axis=0)

X_norm = X.iloc[:, :-4]
X_norm = 2 * (X_norm - X_min) / (X_max - X_min) - 1

X_input = pd.concat([X_norm, X_last_4], axis=1)

input_data = X_input.iloc[:, 2:] 

# Convert to tensor and move to device
X_tensor = torch.tensor(input_data.values, dtype=torch.float32).to(device)

# Get predictions
estimator.eval()
with torch.no_grad():
    predicted_torques = estimator(X_tensor)

predicted_torques_numpy = predicted_torques.cpu().numpy()

# Unnormalize predicted torques
Y_max_tensor = torch.tensor(Y_max.values, dtype=torch.float32).to(device)
Y_min_tensor = torch.tensor(Y_min.values, dtype=torch.float32).to(device)

predicted_torques_unnormalized = ((torch.tensor(predicted_torques_numpy, dtype=torch.float32).to(device) + 1) * (Y_max_tensor - Y_min_tensor)) / 2 + Y_min_tensor

# Store actual torques for comparison
actual_torques_numpy = Y.to_numpy()

# Log actual and predicted torques for each joint at each time step
writer = SummaryWriter('runs/end2end')

num_joints = predicted_torques_numpy.shape[1]  # Get number of joints from predictions
num_time_steps = predicted_torques_numpy.shape[0]  # Get number of time steps

for t in range(num_time_steps):
    for i in range(num_joints):
        writer.add_scalars(f'Joint {i+1}', {
            'Actual Torque': actual_torques_numpy[t, i],  
            'Predicted Torque': predicted_torques_unnormalized[t, i].item()  # Convert tensor to scalar for logging
        }, global_step=t)  

writer.close()  # Close the writer when done

print("Predictions made and logged to TensorBoard.")


/tmp/ipykernel_14477/2629748799.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  estimator = torch.load(model_path, map_location=device)  # Load model to the correct dev

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/userdata/ssajja2s/models/end2end_predictor.pth'

In [ ]:
# Estimator Evaluation

import time
import numpy as np
import torch
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
from NMPCPredictor import NMPCPredictor  # Ensure this path is correct

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model checkpoint
# model_path = '/home/jovyan/userdata/ssajja2s/models/end2end_predictor.pth'
model_path = '/home/jovyan/userdata/ssajja2s/models/estimator_500_epoch.pth'
checkpoint = torch.load(model_path, map_location=device)

# Initialize the model (ensure input and output sizes are correct)
input_size = checkpoint['model_state_dict']['hidden1.weight'].shape[1]
output_size = checkpoint['model_state_dict']['output.weight'].shape[0]
neurons = 2560  # or whatever value you used during training

# Initialize estimator with neurons parameter
estimator = NMPCPredictor(input_size=input_size, output_size=output_size, neurons=neurons).to(device)

# Load the state dict into the model
estimator.load_state_dict(checkpoint['model_state_dict'])

# Load normalization parameters
X_min = checkpoint['x_min']
X_max = checkpoint['x_max']
Y_min = checkpoint['y_min']
Y_max = checkpoint['y_max']

print("Model loaded successfully along with normalization parameters.")

# Load the input data
# data_path = "/home/jovyan/userdata/ssajja2s/data_files/new_data/go1_trot_data_06.csv"
# data_path = "/home/jovyan/userdata/ssajja2s/data_files/eval_data/go1_trot_data_eval_test.csv"
# data_path ="/home/jovyan/userdata/ssajja2s/data_files/eval_data/go1_trot_data_eval.csv"
data_path = "/home/jovyan/userdata/ssajja2s/data_files/eval_data/go1_trot_eval_data_v3.csv"
df = pd.read_csv(data_path)




# Prepare input and output data
obs_state_data = df.iloc[:, :-12]
tau_data = obs_state_data.iloc[:, -12:]


# Drop the first column 'time'
obs_state_data = obs_state_data.drop('time', axis=1)

Y = obs_state_data.iloc[:, :10]
X = obs_state_data.iloc[:, 10:]


# Store the last 4 columns 
X_last_4 = X.iloc[:, -4:]

# Scale the data to be between -1 and 1
X_norm = X.iloc[:, :-4]
X_norm = 2 * (X_norm - X_min) / (X_max - X_min) - 1

X_input = pd.concat([X_norm, X_last_4], axis=1)

input_data = X_input


# Convert to tensor and move to device
X_tensor = torch.tensor(input_data.values, dtype=torch.float32).to(device)

# Get predictions
estimator.eval()
with torch.no_grad():
    predicted_states = estimator(X_tensor)

predicted_states_numpy = predicted_states.cpu().numpy()

# Ensure Y_max and Y_min are NumPy arrays
Y_max = np.array(Y_max)
Y_min = np.array(Y_min)


# Adjusted unnormalization formula using loaded normalization parameters
predicted_states_unnormalized = ((predicted_states_numpy + 1) / 2) * (Y_max.reshape(1, -1) - Y_min.reshape(1, -1)) + Y_min.reshape(1, -1)

# Store actual torques for comparison
actual_states_numpy = Y.to_numpy()

# Log actual and predicted torques for each joint at each time step
writer = SummaryWriter('runs/estimator')

num_states = predicted_states_unnormalized.shape[1]  # Get number of joints from predictions
num_time_steps = predicted_states_unnormalized.shape[0]  # Get number of time steps

state_names = [
    'base_pos_x', 'base_pos_y', 'base_pos_z',
    'base_ori_x', 'base_ori_y', 'base_ori_z', 'base_ori_w',
    'base_vel_x', 'base_vel_y', 'base_vel_z'
]

for t in range(num_time_steps):
    for i in range(num_states):
        writer.add_scalars(state_names[i], {
            'Actual value': actual_states_numpy[t, i],  
            'Predicted value': predicted_states_unnormalized[t, i]  # Use unnormalized predictions for logging
        }, global_step=t)  

writer.close()  # Close the writer when done

print("Predictions made and logged to TensorBoard.")


# Save predicted torques to a CSV file
predicted_states_df = pd.DataFrame(predicted_states_unnormalized, columns=state_names)
predicted_states_df.to_csv('predicted_states.csv', index=False)



In [ ]:
X = obs_state_data.iloc[:, :10]
Y = tau_data

# remove pose_x and pose_y and for X which are first two columns
X = X.iloc[:, 2:]